In [1]:
# Add feature: top x locations per UN region 
# - Intersect GADM layer with UN regions and then loop through regions

# Add features:
# - Satellite images before and after
# - Number of hectares burned
# - Trend of last x days
# - Number of days with at least 1 alert
# - Proportion covered by forest

from utils.dates import date_range_list
from utils.query_api import send_request
from utils.std_score import country_std_score

iso_list = ['IND']
drop_thresh = 30

for country in iso_list:
    dates = date_range_list(days=7)
    data = send_request(country, dates, printq=True)

    if data:
        adm1_out, adm2_out = country_std_score(data, dates)

adm1_out = adm1_out[adm1_out['current_alerts'] > drop_thresh]
adm2_out = adm2_out[adm2_out['current_alerts'] > drop_thresh]



ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.9 from "C:\Users\james.maccarthy\Miniconda3\envs\fires\python.exe"
  * The NumPy version is: "1.20.2"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed while importing _multiarray_umath: The specified module could not be found.


In [2]:
tuple(df[df['current_alerts'] > drop_thresh] for df in country_std_score(data, dates))

(   country  adm1  current_alerts  significance       score
 0      IND    19             403      0.469745  189.307321
 1      IND    34             173      0.134248   23.224841
 2      IND    20              71      0.201727   14.322603
 3      IND    26              66     -0.117397   -7.748216
 4      IND    29              44     -0.423798  -18.647114
 6      IND     5              70      1.042904   73.003271
 9      IND    12             116      1.486864  172.476266
 13     IND     4              34      0.147310    5.008549
 16     IND    23              40      1.670620   66.824797,
     country  adm1  adm2  current_alerts  significance      score
 2       IND    19    40              59      1.341940  79.174446
 8       IND    19    14              39      1.918550  74.823435
 50      IND    34    70              36     -0.127117  -4.576208
 265     IND    12     7              31      2.723597  84.431499
 345     IND     4    17              32      0.314774  10.072763)

In [7]:
def rank_output(df, top_n):
    top_sig = df.sort_values(['significance'], ascending=False).head(top_n)
    top_score = df.sort_values(['score'], ascending=False).head(top_n)
    return top_sig, top_score


In [20]:
# output.py
import time
from datetime import datetime as dt
from pandas import DataFrame, ExcelWriter, read_csv

def rank_output(df, top_n):
    top_sig = df.sort_values(['significance'], ascending=False).head(top_n)
    top_score = df.sort_values(['score'], ascending=False).head(top_n)
    return top_sig, top_score


def write_results(top_sig, top_score, top_n, adm_level):
    print('Writing results...', end="")
    today = dt.today().date()
    filename = f'{adm_level}_top_{top_n}_output_' + today.strftime('%Y%m%d') + '.xlsx'
    writer = ExcelWriter(filename, engine='xlsxwriter')
    top_sig.to_excel(writer, sheet_name='top_significance', index=False)
    top_score.to_excel(writer, sheet_name='top_score', index=False)
    writer.save()

In [21]:
adm1_out = []
regions = df.adm1.unique()
for region in regions:
    adm1_out.append(adm1_significance(df, region))
print(adm1_out[:3])
adm1_df = format_adm1_out(adm1_out, drop_threshold=10)
print(adm1_df.head(3))
top_sig, top_score = rank_output(adm1_df, top_n=20)
print(top_sig.head(3))

[('IND', 19, 1.9486861836047127, 1001), ('IND', 20, 1.0231502599214375, 184), ('IND', 26, -0.5241250675095297, 81)]
  country  adm1  significance  current_alerts        score
0     IND    19      1.948686            1001  1950.634870
1     IND    20      1.023150             184   188.259648
2     IND    26     -0.524125              81   -42.454130
   country  adm1  significance  current_alerts       score
23     IND    13      2.831849              40  113.273959
20     IND    35      2.828880             123  347.952259
8      IND    34      2.794225             199  556.050871


In [22]:
def gadm_countries(file='gadm_adm1.csv'):
    keep_cols = ['GID_0', 'NAME_0', 'GID_1', 'NAME_1']
    gadm_df = read_csv(file, usecols=keep_cols)
    iso_list = gadm_df.GID_0.unique()
    return iso_list

In [11]:
##### End of demo #####

In [12]:
# iso_list = ['COD', 'IND']
iso_list = gadm_countries()

start = dt.now()
print(f"Start: {start}")

out = all_adm1_significance(iso_list, days=7)

end = dt.now()
print(f"End: {end}")
print(f"Total time: {end - start}")

Start: 2021-04-08 12:49:53.935526
AFG


NameError: name 'create_index' is not defined

In [ ]:
out[0].head()

In [113]:
top_sig, top_score = rank_output(out[0], top_n=20)
write_results(top_sig, top_score, top_n=20, adm_level='adm1')
print("done")
top_sig, top_score = rank_output(out[1], top_n=20)
write_results(top_sig, top_score, top_n=20, adm_level='adm2')
print("done")

Writing results...done
Writing results...done


In [ ]:
# Add trend of last x days
# Add number of days with at least 1 alert
# Add proportion covered by forest